<a href="https://colab.research.google.com/github/abbiu/School/blob/main/BAIT_508_Indsutry_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load data**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Loading the data from Google Drive
 # **** (cite AI usage!) ChatGPT 5 - on changing Google Drive url for loading
# data
url_mg = 'https://drive.google.com/uc?export=download&id=1jinQrkO-FevyrUlFDEV2QJiKtTtEjzrt'
url_pf = 'https://drive.google.com/uc?export=download&id=1-gbc5tMK6t1BB9_yFYMo4YIySHMNtGkq'


In [6]:
df_mg = pd.read_csv(url_mg)
df_pf = pd.read_csv(url_pf)


In [ ]:
df_mg.head()

In [ ]:
df_pf.head()

# **1 Quantitative Analysis of the Food Store Industry**

## **1.1 Data Filtering**

In [ ]:
df_pf_final = df_pf[df_pf["sic"].astype(str).str.startswith('54', na=False)]
display(df_pf_final.head())

In [ ]:
print(df_pf_final.isna().sum())

In [ ]:
# a. Number of unique firm-year observations
num_firm_year = df_pf_final.groupby(['gvkey', 'fyear']).size().shape[0]
print("Number of unique firm-year observations:", num_firm_year)

# b. Number of unique firms
num_firms = df_pf_final.groupby('gvkey').ngroups
print("Number of unique firms:", num_firms)

# c. Number of firms with records over all 27 years (1994-2020)
# Count how many years each firm appears
years_per_firm = (
    df_pf_final[['gvkey', 'fyear']]
    .drop_duplicates()
    .groupby('gvkey')['fyear']
    .count()
)
print(years_per_firm)
# Firms that have all 27 years
firms_all_years = years_per_firm[years_per_firm == 27].count()
print("Number of firms with records for all 27 years:", firms_all_years)

## **1.2 Preliminary Analysis**

### Question 1
 What are the top 10 firms with the highest stock price?

In [ ]:
df_pf_final[df_pf_final["fyear"] == 2020].sort_values(by="prcc_c", ascending=False).head(10)


### Question 2
 What are the top 10 firms with the highest sales (column "sale") in the entire history of the dataset?

In [ ]:
df_pf.sort_values(by="sale", ascending=False).head(10)

#### Question 3
 What is the geographical distribution (column "location") of all the firms? In other words, how many firms are there in each location? Please list the top 10 locations

In [ ]:
df_pf_final.groupby("location").count().sort_values(by="location", ascending=False).head(10)

#### Question 4

Create a line chart to show the average stock price (column "prcc_c") in the selected sector(s) across the years. If you have selected multiple sectors, draw multiple lines to show them separately.

In [ ]:
mean_df_pf = df_pf_final.groupby(by='fyear').mean(numeric_only=True).reset_index()
mean_df_pf.head()

In [ ]:
# Please execute the cell above first to define `mean_df_pf`
plt.plot(mean_df_pf['fyear'],mean_df_pf['prcc_c'])
plt.xlabel('Year')
plt.ylabel('Average Stock Price')
plt.title('Average Stock Price in the Food Store Industry over Time')
plt.show()

#### Quesiton 5
Which firm was affected the most by the 2008 Financial Crisis, as measured by the percentage drop in stock price from 2007 to 2008?

In [ ]:
df_pf_07_08 =  df_pf_final[(df_pf_final['fyear']==2007) | (df_pf_final['fyear']==2008)]

In [ ]:
 #reshape the dataframe
stock_price_df = df_pf_07_08.pivot(index='conm',columns = 'fyear', values = 'prcc_c').reset_index()
stock_price_df['change_percentage'] = (stock_price_df[2008]-stock_price_df[2007])/stock_price_df[2007]
stock_price_df.head()

In [ ]:
#find the company that was affected the most
stock_price_df.sort_values('change_percentage').iloc[0,0]

#### Quesiton 6
Plot the average Return on Assets (ROA) for the firms located in the “USA” across the years. ROA is calculated as ni/asset.

In [ ]:
df_pf_final['location'].unique()

In [ ]:
USA_df = df_pf_final[df_pf_final['location']=='USA']
USA_df.head()
# roa already exists in the given data frame
#USA_df['ROA'] = USA_df['ni']/USA_df['asset']

In [ ]:
roa_USA_df = USA_df.groupby('fyear').mean(numeric_only = True).reset_index()[['fyear','roa']]

In [ ]:
plt.plot(roa_USA_df['fyear'],roa_USA_df['roa'])
plt.xlabel('Year')
plt.ylabel('Average ROA')
plt.title('Average ROA of the US Food Store Industry over Time')
plt.show()

# **2 Text Analysis on the Food Store Industry**

## **2.1 Text Cleaning**

In [ ]:
## !!! Cite AI usage for using drop box link to load large size data - ChatGPT 5
url_10k = 'https://www.dropbox.com/scl/fi/8p9klgbjasek74xfqaydi/2020_10K_item1_full.csv?rlkey=mvsehcwgkevlt5t3fqjnirt2e&st=oyy6365c&dl=1'

In [ ]:
df_10k = pd.read_csv(url_10k)

In [ ]:
df_10k.head()

Cleaning the texts and store the new values in column *item_1*

In [ ]:
#!!! Cite, code directly copied from NLP Part 2 Keyword Analysis Lecture notebook !!!
translator = str.maketrans('', '', string.punctuation)
sw = stopwords.words('english')

def clean_text(text):
    ''' This function takes a string as input and
        returns a cleaned version of the string
        Specifically, it makes the string into lower case and remove punctuations
    '''
    text_lower = text.lower() # make it lowercase
    text_no_punctuation = text_lower.translate(translator) # remove punctuation
    clean_words = [w for w in text_no_punctuation.split() if w not in sw] # remove stopwords
    return ' '.join(clean_words)

In [ ]:
# be careful with re-running this, took me around 3 minutes to execute
df_10k['item_1'] = df_10k['item_1_text'].apply(clean_text)

In [ ]:
df_10k.head()

## **2.2 Keyword Analysis**

###Create a new DataFrame that includes only firms in your selected industry sector(s). Ensure that you merge the 10-K data with the previous "public_firm.csv" data using an inner join.

> Add blockquote



In [ ]:
# inner join 2 datasets... leaving the rest to you guys!
df_sector = pd.merge (df_10k, df_pf_final, left_on=['gvkey', 'year'], right_on=['gvkey','fyear'], how='inner')


# Step 1: Select sector(s)
#df_sector = df_merged[df_merged['sic'].astype(str).str.startswith('54')]
#df_sector = df_sector[df_sector['item_1'].str.strip() != ""].reset_index(drop=True)


In [ ]:
#df_sector.drop_duplicates(subset=['gvkey'], inplace=True)
#df_sector['name']


###Generate the top 10 keywords for each firm based on two different methods: word counts and TF-IDF score.

In [ ]:
from collections import Counter

def get_keywords_counts(document_list, top_n=10):
    """
    Get top keywords for each document using raw word counts.
    Returns:
    - list of dicts: {"words": "word1 word2 ... word10", "counts": [(word, count), ...]}
    """
    results = []
    for text in document_list:
        words = text.split()  # simple tokenization
        c = Counter(words)
        top_pairs = c.most_common(top_n)
        top_words = ' '.join([w for w, _ in top_pairs])

        results.append({"words": top_words, "counts": top_pairs})
    return results

# Apply to your sector
documents = df_sector['item_1'].fillna("").tolist()
df_sector['count_results'] = get_keywords_counts(documents, top_n=10)

# Just words for easier viewing
df_sector['top_keywords_counts'] = df_sector['count_results'].apply(lambda x: x['words'])

# Show sample firms with their top keywords (by raw counts)
pd.set_option('display.max_colwidth', None)
display(df_sector[['name', 'gvkey', 'sic', 'top_keywords_counts']].head(13))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_keywords_tfidf(document_list, top_n=10):
    """
    Get top keywords for each document using TF-IDF.
    Returns:
    - list of dicts: {"words": "word1 word2 ... word10", "scores": [(word, score), ...]}
    """
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(document_list)
    feature_names = vectorizer.get_feature_names_out()

    results = []
    for i in range(len(document_list)):
        row = tfidf_matrix[i, :].tocoo()
        tfidf_scores = list(zip(row.col, row.data))
        sorted_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

        # Top N words and scores
        top_pairs = [(feature_names[idx], float(score)) for idx, score in sorted_scores[:top_n]]
        top_words = ' '.join([w for w, _ in top_pairs])

        results.append({"words": top_words, "scores": top_pairs})

    return results

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
get_keywords_tfidf(corpus)


# Apply
documents = df_sector['item_1'].fillna("").tolist()
df_sector['tfidf_results'] = get_keywords_tfidf(documents, top_n=10)

# Show just words in the main view
df_sector['top_keywords_tfidf'] = df_sector['tfidf_results'].apply(lambda x: x['words'])

# See first few firms with keywords
pd.set_option('display.max_colwidth', None)
display(df_sector[['name', 'gvkey', 'sic', 'top_keywords_tfidf']].head(13))


In [ ]:
# Gene
def get_keywords_tfidf(document_list):
    '''
    This function gets a list of documents as input and returns a list of top 10 keywords for each document using TF-IDF scores.
    Input: A list of documents (text)
    Output: The corresponding top 10 keywords for each document based on tf-idf values
    '''
    vectorizer = TfidfVectorizer() # Step 1: Create a TF-IDF vectorizer
    tfidf_matrix = vectorizer.fit_transform(document_list) # Step 2: Calculate the TF-IDF matrix
    feature_names = vectorizer.get_feature_names_out() # Step 3: Get feature names (words)

    # Step 4: Extract top 10 keywords for each document
    top_keywords = [] # accumulator
    for i in range(len(document_list)):
        feature_index = tfidf_matrix[i, :].nonzero()[1]
        feature_value = [tfidf_matrix[i, x] for x in feature_index]
        tfidf_scores = zip(feature_index, feature_value)
        sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
        top_keywords.append(' '.join([feature_names[i] for i, _ in sorted_tfidf_scores[:10]]))

        if i % 200 == 199:
            print(f'Processed {i+1}/{len(document_list)} documents.')

    return top_keywords

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
get_keywords_tfidf(corpus)

docs = df_sector['item_1'].tolist()
len(docs)

# This process will take several minutes.
tfidf_keywords = get_keywords_tfidf(docs)

# add a new column in the dataframe
df_sector['top_keyword_tfidf'] = tfidf_keywords

# check what tfidf_keywords contain.
print(type(tfidf_keywords))
print(len(tfidf_keywords))


In [ ]:
tfidf_keywords

In [ ]:
# Pick which firm to analyze
n = 8  # change this index to any firm number

# Get firm info
firm_name = df_sector['name'].iloc[n]
gvkey = df_sector['gvkey'].iloc[n]

# Show the words
print(f"Top 10 words for {firm_name} (gvkey={gvkey}):")
print(df_sector['top_keywords_tfidf'].iloc[n])

# Dig into the dict to see scores
print("\nWith scores:")
for word, score in df_sector['tfidf_results'].iloc[n]['scores']:
    print(f"{word}: {score:.4f}")

###Create two wordclouds to visualize the keywords across all firms in the selected sector(s): one based on the word counts and another based on the TF-IDF scores.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Build one big string from the text column
text1 = ' '.join(keyworddf['top_keywords_counts'].astype(str).tolist())

wc = WordCloud(width=800, height=400, max_font_size=100, background_color='white')
wordcloud1 = wc.generate(text1)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud : Sector-Level Word Counts", fontsize=16, pad=10)
plt.tight_layout()
plt.savefig('keyword_all.png', dpi=200, bbox_inches='tight')
plt.show()


In [ ]:
# import relavant packages
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# prepare text# prepare text
text1 = ' '.join(df_sector['item_1'].tolist())
# lower max_font_size
wordcloud1 = WordCloud(width=800, height=400, max_font_size=100, background_color='white').generate(text1) # note that text is a string, not a list

plt.figure(figsize=(10,5))
plt.title("WordCloud : Sector-Level Word Counts", fontsize=16)
plt.axis('off')
plt.imshow(wordcloud1)
plt.savefig('keyword_all.png') # save as PNG file
plt.show()

In [ ]:
# --- prepare documents (one doc per firm/row) ---
texts = df_sector['item_1'].dropna().astype(str).tolist()

# --- TF-IDF: build term weights across the sector ---
# You can tweak: stop_words, token_pattern, ngram_range, max_features
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english',              # remove common English words
    token_pattern=r'(?u)\b[a-zA-Z]{3,}\b',  # words with >=3 letters
    ngram_range=(1, 1),                # unigrams; set to (1,2) to include bigrams
    max_features=8000                  # cap vocab size (optional)
)
X = vectorizer.fit_transform(texts)        # shape: (n_docs, n_terms)
terms = vectorizer.get_feature_names_out()

# Sum TF-IDF across all documents to get sector-level importance per term
sector_tfidf = np.asarray(X.sum(axis=0)).ravel()   # shape: (n_terms,)
freqs = dict(zip(terms, sector_tfidf))

# (Optional) keep only the top-N weighted terms for a cleaner cloud
N = 1000
freqs = dict(sorted(freqs.items(), key=lambda kv: kv[1], reverse=True)[:N])

# --- WordCloud from TF-IDF frequencies ---
wordcloud1 = WordCloud(
    width=800,
    height=400,
    max_font_size=100,
    background_color='white',
    prefer_horizontal=0.9,
    normalize_plurals=False
).generate_from_frequencies(freqs)

# --- plot & save ---
plt.figure(figsize=(10, 5))
plt.axis('off')
plt.imshow(wordcloud1, interpolation='bilinear')
plt.savefig('keyword_all_tfidf.png', bbox_inches='tight', dpi=150)
plt.show()


## **2.3 Word Embedding**



### **2.3.1 Importing and descriptive analytics**

#### Install and import gensim

In [ ]:
#!pip3 install gensim
!pip uninstall -y gensim numpy
!pip install gensim==4.3.3 numpy==1.26.4

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# list of list of words
docs = [row.split() for row in df_10k['item_1']]

In [ ]:
print(len(docs))

#### Training and load word2vec model

In [ ]:
# training word2vec model using the list of words in `sent`
model = Word2Vec(docs, min_count=5, vector_size=50, workers=3, window=5, sg = 1)

In [ ]:
# save the model for future use; you don't need to train Word2Vec for multiple times
model.save("word2vec.model")

In [ ]:
# load model from stored file
model = Word2Vec.load("word2vec.model")

### **2.3.2 Word similarity**


In [ ]:
model.wv.most_similar('company')

- shares, chief, officer, directors seems to be the most relevant to company, which makes sense.
- 7, 2016, 1000 suggests that they are oftenly mentioned in the report like "the company year 2016" or "1000 shares owned", etc.
- the model think that these words are relevant to "company" at > 0.95 score

In [ ]:
model.wv.most_similar('store')

In [ ]:
model.wv.most_similar('customers')

Most similar words

In [ ]:
model.wv.most_similar('company')

In [ ]:
model.wv.most_similar('store')

In [ ]:
model.wv.most_similar('customers')

# **3. Comprehensive Analysis of One Sample Firm**




Let say we picked CASEYS GENERAL STORES INC

## Option 1 -  Find the focal firm’s competing firms

### Set up DocumentSimilarity

In [ ]:
class DocumentSimilarity:

    def __init__(self, model, gvkeys, conm, keywordslist):
        '''
        Initialize the class
        model: the word2vec model
        gvkeys: a list/pandas series of unique firm identifiers
        conm: a list/pandas series of company names
        keywordslist: a list of keywords

        gvkeys and keywordslist should be of the same length
        '''

        assert len(gvkeys) == len(keywordslist) == len(conm), "gvkeys, conm, keywordslist should should be of the same length"

        # store the information
        self.model = model
        self.firms = list(gvkeys)
        self.conm = list(conm)
        self.keywordslist = [x.split() for x in list(keywordslist)]

        # generate document embedding
        self.document_embeddings = [self.model.wv.get_mean_vector(x) for x in self.keywordslist]

        # convert to array to facilitate computation, normalize it
        self.document_array = np.array(self.document_embeddings)
        self.document_array = self.document_array / np.linalg.norm(self.document_array, axis=1)[:, np.newaxis]

    def get_firm_embedding(self, firm):
        '''Given the firm unique identifier, return the embedding of this firm'''

        return self.document_embeddings[self.firms.index(firm)]

    def similarity(self, firm1, firm2):
        '''Given two firms' unique identifiers, return the similarity between the two firms'''
        firm1 = self.document_embeddings[self.firms.index(firm1)]
        firm2 = self.document_embeddings[self.firms.index(firm2)]

        return np.dot(firm1, firm2) / (np.linalg.norm(firm1) * np.linalg.norm(firm2))

    def most_similar(self, firm, topn = 5):
        '''Given one firm unique identifier, return the topn similar firms to it
        firm: firm unique identifier
        topn: the number of firms to return
        '''

        v = self.document_embeddings[self.firms.index(firm)]
        v = v / np.linalg.norm(v)

        cosine_similarities = np.dot(self.document_array, v)

        # find the index of the top n companies
        sorted_indices = np.argsort(-cosine_similarities)
        largest_n_indices = sorted_indices[:topn + 1]

        return [(self.firms[x], self.conm[x], cosine_similarities[x]) for x in largest_n_indices[1:]]

In [ ]:
# Maybe changing this to the final merge df to get the most similar from the whole dataset(*but collab gonna crash :< )
docsim = DocumentSimilarity(model = model, gvkeys=df_sector['gvkey'], conm = df_sector['name'],
                       keywordslist = df_sector['top_keywords_tfidf'])
type(docsim)

In [ ]:
df_sector[['gvkey','name']].drop_duplicates().head(10)

### Find the most similar firms to CASEYS GENERAL STORES INC

In [ ]:
# CASEYS GENERAL STORES INC (gvkey = 2807)
docsim.get_firm_embedding(firm = 2807)

In [ ]:
docsim.most_similar(firm = 2807, topn = 10)

## Option 2 - Descriptive analytics of the chosen firm

In [ ]:
df_chosen = df_pf_final[df_pf_final['gvkey']==2807]

In [ ]:
df_chosen.info()

In [ ]:
df_chosen.corr(numeric_only=True)

In [ ]:
plt.plot(df_chosen['fyear'],df_chosen['prcc_c'])
plt.xlabel('Year')
plt.ylabel('Stock price')
plt.title('Stock price movement by year')
plt.show()

- we see that net income(ni), asset and sale has strong correlation to the stock price.

In [ ]:
plt.plot(df_chosen['fyear'],df_chosen['ni'])
plt.xlabel('Year')
plt.ylabel('Net Income')
plt.title('Net Income by year')
plt.show()

In [ ]:
plt.plot(df_chosen['fyear'],df_chosen['roa'])
plt.xlabel('Year')
plt.ylabel('Return on Asset')
plt.title('ROA by year')
plt.show()

## Option 3 - Measure firm liquility

Assume that ch(Cash & Short- Term Investment) and Asset are reported in millions:

$$
\text{Liquidity} = \frac{\text{Cash & Short-Term Investments (ch)}}{\text{Total Assets}}
$$

In [ ]:
df_chosen['liquidity'] = df_chosen['ch'].values/df_chosen['asset'].values


In [ ]:
df_chosen['liquidity'].describe()

- on average, on 4% of total assets were in cash & short-term investment
- the highest cash ratio on assets is 12.67%
- the lowest is 0.7%


In [ ]:
plt.plot(df_chosen['fyear'],df_chosen['liquidity'])
plt.xlabel('Year')
plt.ylabel('Liquidity ratio')
plt.title('Liquidity power over the year')
plt.show()

### Testing to be revised
1. **1994–2000:** Liquidity ratio remains very low (<0.03), with small fluctuations. The firm likely reinvested heavily in operations rather than holding large cash reserves.  
2. **2001–2007:** Clear upward trend, peaking around 0.12–0.13 before 2008. Suggests precautionary cash accumulation leading up to the financial crisis.  
3. **2008–2010:** Sharp decline in liquidity. Indicates the firm may have drawn down cash buffers or that assets grew faster than cash during the global financial crisis.  
4. **2011–2016:** Volatile liquidity (0.02–0.05). Reflects unstable cash management, possibly due to uneven profitability, investments, or financing needs.  
5. **2017–2020:** Recovery toward 0.07–0.08, showing the firm rebuilt its liquidity position and stabilized operations.




In [ ]:
df_chosen[['liquidity','prcc_c','roa']].corr(method='pearson')

In [ ]:
# Testing, remove if necessary
# Normalize each variable to 0–1 scale for comparability
df_plot = df_chosen.copy()
df_plot['liq_norm'] = df_plot['liquidity'] / df_plot['liquidity'].max()
df_plot['roa_norm'] = df_plot['roa'] / df_plot['roa'].max()
df_plot['price_norm'] = df_plot['prcc_c'] / df_plot['prcc_c'].max()

# Plot
plt.figure(figsize=(10,6))
plt.plot(df_plot['fyear'], df_plot['liq_norm'], marker='o', label='Liquidity Ratio')
plt.plot(df_plot['fyear'], df_plot['roa_norm'], marker='s', label='ROA')
plt.plot(df_plot['fyear'], df_plot['price_norm'], marker='^', label='Stock Price')

plt.title("Liquidity vs ROA and Stock Price (Normalized) Over Time")
plt.xlabel("Year")
plt.ylabel("Normalized Value (0–1)")
plt.legend()
plt.grid(True)
plt.show()
